In [2]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import plotly

project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.append(project_root)

from preprocessing.video_tracking import read_metrica_event_data
from supportFolder.statical_eventTracking import compute_statistics
from supportFolder.plot_pitch import create_soccer_Pitch
from supportFolder.footBall_amination import FootballAnimator

from supportFolder.statical_eventTracking import build_event_tracking, select_hightlight_events, pair_event_phases
from supportFolder.cralw_eventData import crawl_matchEvent_data

#Hiển thị toàn bộ thông tin cột dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


"Manchester City - Manchester United": 2500045, #England (ngau nhien)  
"Portugal - France": 1694440, #European Champion League (National) (vô dich giai dau)  
"Monaco - Olympique Lyonnais": 2500920, #Ligue 1 (France) (duoc danh gia tot nhung lai da kem)  
"Bayern Mucnchen - Stuttgart": 2517036, #Germany (doi bong noi tieng)  
"SPAL - Sampdoria": 2576337, #Germany (da tot mua giai khi bi danh gia kem)  
"Barcelona - Real Madrid": 2565907, #Spain Doi thi dau giu duoc thu hang tot nhat  
"France - Croatia": 2058017 # Doi vo dich world cup 2018 Mbappé toa sang trong thi dau  


In [ ]:
matchID = 2058017
crawl_matchEvent_data(matchID, saved=True)


 Lấy thông tin dữ liệu cho trận đấu matchID = 2058017

 Thu thập thông tin từ CSDL hệ thống thành công.
Đã lưu thành công file: /home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/data_extractions/event_tracking_raw_2058017.parquet (1459 row)
Đã lưu thành công file: /home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/data_extractions/event_tags_raw_2058017.parquet (2115 row)
Đã lưu thành công file: /home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/data_extractions/formation_timeline_raw_2058017.parquet (45 row)


In [34]:
matchID = 2500045
df_events, df_formations = read_metrica_event_data(game=matchID, saved=True)
df_events.head()


____Xử lý dữ liệu trận: 2500045___


,ID,matchID,matchPeriod,eventSec,eventSecEnd,startFrame,endFrame,eventName,subEventName,teamID,Team,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerID,playerName,playerPosition,toPlayerID,toPlayerName,homeTeamID,awayTeamID,accurate,Goal,ownGoal
0,552130,2500045,1H,2.036874,3.236874,50,80,Pass,Simple pass,1611,Away,52.50,34.00,46.20,47.60,7934,J. Lingard,Midfielder,70122.0,N. Matić,1625,1611,1,0,0
1,552131,2500045,1H,7.181832,11.786454,179,294,Pass,Simple pass,1611,Away,46.20,47.60,29.40,60.52,70122,N. Matić,Midfielder,7938.0,A. Valencia,1625,1611,1,0,0
2,552132,2500045,1H,11.786454,12.986454,294,324,Pass,Simple pass,1611,Away,29.40,60.52,9.45,40.12,7938,A. Valencia,Defender,7910.0,David de Gea,1625,1611,1,0,0
3,552133,2500045,1H,20.541311,23.179631,513,579,Pass,Launch,1611,Away,9.45,40.12,71.40,51.00,7910,David de Gea,Goalkeeper,105339.0,Fernandinho,1625,1611,1,0,0
4,552134,2500045,1H,23.179631,23.227421,579,580,Duel,Ground loose ball duel,1625,Home,33.60,17.00,35.70,13.60,105339,Fernandinho,Midfielder,7905.0,R. Lukaku,1625,1611,0,0,0


In [12]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1497 non-null   int64  
 1   matchID           1497 non-null   int64  
 2   matchPeriod       1497 non-null   object 
 3   eventSec          1497 non-null   float64
 4   eventSecEnd       1497 non-null   float64
 5   startFrame        1497 non-null   int64  
 6   endFrame          1497 non-null   int64  
 7   eventName         1497 non-null   object 
 8   subEventName      1497 non-null   object 
 9   teamID            1497 non-null   int64  
 10  Team              1497 non-null   object 
 11  posBeforeXMeters  1497 non-null   float64
 12  posBeforeYMeters  1497 non-null   float64
 13  posAfterXMeters   1497 non-null   float64
 14  posAfterYMeters   1497 non-null   float64
 15  playerID          1497 non-null   int64  
 16  playerName        1497 non-null   object 


In [35]:
df_game = compute_statistics(df_events, group_col='team', drop_KPIS=["centroid", "totalAccuratePasses"])

df_game.head()

,teamID,nbMatches,totalPasses,shareAccuratePasses,meanPassLength,totalShots,totalGoals,totalDuels
0,1611,1,289,82.35,19.336817,5,3.0,166
1,1625,1,570,88.07,20.284956,16,2.0,196


In [14]:
df_startGame = compute_statistics(df_events, group_col='player', drop_KPIS=["totalAccuratePasses"], df_formations=df_formations)

df_startTeam1 = df_startGame[(df_startGame["teamID"] == 3204) & (df_startGame['lineup']==1)].copy()

plot_title = "Vị trí trung bình của các cầu thủ đội nhà"

pitch_startGame = create_soccer_Pitch(theme="classic")

pitch_startGame.add_position_plot(df_startTeam1, title=plot_title, colour_kpi="totalPasses")


In [15]:
plot_title = "Vị trí trung bình của các cầu thủ đội nhà trong 90 phút thi đấu"

pitch_startGame90 = create_soccer_Pitch(theme="classic")

pitch_startGame90.add_position_plot(df_startTeam1, title=plot_title, colour_kpi="totalPasses90")


In [16]:
fig_pass = create_soccer_Pitch(theme="classic")

df_pass = fig_pass.prepare_pass_(df_events, df_startTeam1, show_top_K_percent=80, view_90_minute=True)

plot_title_pass = "Đường chuyền đội nhà trong 90 phút"

fig_pass.add_position_plot(df_startTeam1, df_passes= df_pass, title=plot_title_pass, colour_kpi="totalPasses90")

fig_pass.show()

In [17]:
df_startTeam2 = df_startGame[(df_startGame["teamID"] == 3164) & (df_startGame['lineup']==1)].copy()

fig_pass = create_soccer_Pitch(theme="classic")

df_pass2 = fig_pass.prepare_pass_(df_events, df_startTeam2, show_top_K_percent=80, view_90_minute=True)

plot_title_pass = "Đường chuyền đội khách trong 90 phút"

fig_pass.add_position_plot(df_startTeam2, df_passes= df_pass2, title=plot_title_pass, colour_kpi="totalPasses90")

fig_pass.show()

In [36]:
goal_phases = select_hightlight_events(
    df_events=df_events,
    event_types="LEAD_TO_GOAL",
    preSec=15,
    postSec=2,
    allowed_events=["Pass", "Shot", "Duel"]
)

reviewed_phases = {}   # key: valid_phase_idx, value: tracking_df

MIN_FRAMES = 100
valid_phase_idx = 0

for i, phase_df in enumerate(goal_phases):
    print(f"▶ Processing phase {i+1}/{len(goal_phases)}")

    tracking_df = build_event_tracking(
        df_events=phase_df,
        event_ids=phase_df["ID"].tolist()
    )

    if tracking_df.empty:
        continue

    n_frames = tracking_df["frame"].nunique()
    if n_frames < MIN_FRAMES:
        print(f"   ⏭ Skip (only {n_frames} frames)")
        continue

    valid_phase_idx += 1
    print(f"   ✅ Render phase {valid_phase_idx} ({n_frames} frames)")

    animator = FootballAnimator(tracking_df)
    fig = animator.build_animation()
    fig.show()

    # ⭐ QUAN TRỌNG: lưu phase đã review
    reviewed_phases[valid_phase_idx] = tracking_df

▶ Processing phase 1/9
   ⏭ Skip (only 50 frames)
▶ Processing phase 2/9
   ⏭ Skip (only 25 frames)
▶ Processing phase 3/9
   ✅ Render phase 1 (150 frames)


▶ Processing phase 4/9
   ⏭ Skip (only 50 frames)
▶ Processing phase 5/9
   ✅ Render phase 2 (225 frames)


▶ Processing phase 6/9
   ⏭ Skip (only 25 frames)
▶ Processing phase 7/9
   ✅ Render phase 3 (200 frames)


▶ Processing phase 8/9
   ⏭ Skip (only 50 frames)
▶ Processing phase 9/9
   ⏭ Skip (only 25 frames)


In [37]:
from supportFolder.video_export import MPLFootballAnimator

PHASES_TO_SAVE = [1, 2, 3]

for phase_idx in PHASES_TO_SAVE:
    tracking_df = reviewed_phases[phase_idx]

    animator = MPLFootballAnimator(
        tracking_df=tracking_df,
        fps=10,
        matchID=matchID,
        phase_idx=phase_idx
    )

    animator.save_mp4() 


✅ Saved MP4: /home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/video_/match_2500045_phase_1_Home.mp4
✅ Saved MP4: /home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/video_/match_2500045_phase_2_Away.mp4
✅ Saved MP4: /home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/video_/match_2500045_phase_3_Away.mp4
